In [2]:
# import the dependencies
import numpy as np

#### [Problem 1] Creating a one-dimensional convolutional layer class that limits the number of channels to one

In [12]:
class SimpleConv1d():

  def forward(self, X, W, b):
    a = []
    for i in range(len(W)-1):
      a.append((np.matmul(X[i:i+len(W)],W))+b[0])
    return np.array(a)

  def backward(self, X, W, dA):
    db = np.sum(dA)
    dW = []
    for i in range(len(W)):
      dW.append(np.matmul(dA,X[i:i+len(dA)]))
    dW = np.array(dW)
    dX = []
    new_W = np.insert(W[::-1],0,0)
    new_W = np.append(new_W,0)
    for i in range(len(new_W)-1):
      dX.append(np.matmul(new_W[i:i+len(dA)],dA))
    dX = np.array(dX[::-1])
    return db, dW, dX

#### [Problem 2] Output size calculation after one-dimensional convolution

In [23]:
def output_size_calc(input_size, f, p=0, s=1):
  out_size = ((input_size+2*p-f)/s)+1
  return int(out_size)

#### [Problem 3] Experiment of one-dimensional convolutional layer with small array

In [7]:
X = np.array([1,2,3,4])
W = np.array([3, 5, 7])
b = np.array([1])
delta_a = np.array([10, 20])

In [13]:
s1dconv = SimpleConv1d()
forward_prop = s1dconv.forward(X, W, b)
db, dW, dX = s1dconv.backward(X, W, delta_a)
print(forward_prop)
print(db)
print(dW)
print(dX)

[35 50]
30
[ 50  80 110]
[ 30 110 170 140]


##### [Problem 4] Creating a one-dimensional convolutional layer class that does not limit the number of channels


In [14]:
class ConvSimpleInitializer:

    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, shape):
        W = self.sigma * np.random.randn(*shape)
        return W
    
    def b(self, shape):
        b = self.sigma * np.random.randn(*shape)
        return b

In [17]:
class ConvSGD:
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
      layer.W -= self.lr * layer.dW / len(layer.Z)
      layer.b -= self.lr * layer.db / len(layer.Z)

In [18]:
class Adagrad:
  def __init__(self,lr):
    self.lr = lr
    self.hW = 0
    self.hb = 0

  def update(self, layer):
    self.hW += layer.dW*layer.dW
    self.hb += layer.db*layer.db
    layer.W -= self.lr * layer.dW
    layer.b -= self.lr * layer.db
    return layer

In [19]:
# mini batch
class GetMiniBatch:
    """
Iterator to get a mini-batch
    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [ ]:
class Conv1d:
  def __init__(self, batch_size, input_size_channel=1, output_size_channel=1, padding=0, initializer, optimizer):
    self.batch_size = batch_size
    self.input_size_channel = input_size_channel
    self.output_size_channel = output_size_channel
    self.padding = padding
    self.initializer =  initializer
    self.optimizer = optimizer
    self.W = initializer.W(input_size_channel, output_size_channel, batch_size)
    self.b = initializer.b(output_size_channel)
    pass

  def forward(self, X):
    self.input_size = X.shape[-1]
    self.output_size = output_size_calc(self.input_size, self.batch_size, self.padding)
    X = X.reshape(self.input_size_channel, self.input_size)
    self.X = np.pad(X, (0,0),(self.batch_size-1),0)
    self.X_1 = np.zeros((self.input_size_channel, self.batch_size, self.input_size + (self.batch_size - 1)))
    for i in range(self.batch_size):
      self.X_1[:, i] = np.roll(self.X, -i, axis=-1)
    A = np.sum(self.X_1[:, :, self.batch_size-1-self.padding:self.input_size + self.padding]self.W[:,:,:,np.newaxis],axis(1,2)+self.b.reshape(-1,1))
    return A